In [ ]:
import pandas as pd
import folium
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from branca.colormap import LinearColormap
import json

# Loading data of model variables, locations of data centers and the census tract shapfile
model_vars = pd.read_csv("model_vars.csv")
atlanta_locations = pd.read_csv("atlanta_locations.csv")
with open("tl_2020_13_tract.json", 'r') as f:
    geojson_data = json.load(f)

# Fix FIPS format to match GEOID (ensure 11-digit string)
model_vars['FIPS'] = model_vars['FIPS'].astype(str).str.zfill(11)

# Prepare PC1 and scale
numeric = model_vars.select_dtypes(include='number').copy()
scaled = StandardScaler().fit_transform(numeric)
model_vars['PC1'] = PCA(n_components=2).fit_transform(scaled)[:, 0]
model_vars['PC1_scaled'] = MinMaxScaler().fit_transform(model_vars[['PC1']])

# Create base map
m = folium.Map(location=[32.9, -83.5], zoom_start=7, tiles=None, control_scale=True)

# Add light base tiles
folium.TileLayer(
    tiles="cartodbpositron",
    name="Light Basemap",
    control=True
).add_to(m)

# PC1 Colormap
pc1_colormap = LinearColormap(
    colors=['#ffffcc', '#ffeda0', '#feb24c', '#f03b20', '#bd0026'],
    index=[0.0, 0.25, 0.5, 0.75, 1.0],
    vmin=0, vmax=1,
    caption='Composite Burden Score (PC1 - Scaled)'
)

# Map PC1 values into GeoJSON features
geojson_map = {row['FIPS']: row['PC1_scaled'] for _, row in model_vars.iterrows()}
for feature in geojson_data['features']:
    geoid = feature['properties']['GEOID']
    feature['properties']['PC1_scaled'] = geojson_map.get(geoid, None)

# Add GeoJson layer
folium.GeoJson(
    geojson_data,
    name="PC1 Composite Risk",
    style_function=lambda feature: {
        'fillColor': pc1_colormap(feature['properties']['PC1_scaled']) if feature['properties']['PC1_scaled'] is not None else 'gray',
        'color': 'black',
        'weight': 0.3,
        'fillOpacity': 0.75,
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['GEOID', 'PC1_scaled'],
        aliases=['Tract FIPS:', 'PC1 (scaled):'],
        localize=True
    )
).add_to(m)

# Add custom legend
pc1_colormap.add_to(m)

# Add data center markers
for _, row in atlanta_locations.iterrows():
    popup = f"<b>{row['Name']}</b><br>Company: {row['Company']}<br>Address: {row['Address']}"
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=popup,
        tooltip=row['Company']
    ).add_to(m)

#  Layer control
folium.LayerControl(collapsed=False).add_to(m)

# Show map
m